# Pair Usage and Event Tables Example

In [1]:
# Prepare example data set
usages = [
 {
  'start_time': 10,
  'end_time' : 15,
  'collection_id': 1,
  'instance_index': 2,
  'machine_id' : 3,
  'alloc_collection_id' : 4,
  'alloc_instance_index' : 5,
  'collection_type' : 6,
  'average_usage' : {'cpus':0.7, 'memory':8},
  'maximum_usage' : {'cpus':0.9, 'memory':10},
  'random_sample_usage' : {'cpus':0.11, 'memory':12},
  'assigned_memory' : 13,
  'sample_rate' : 17,
  'cpu_usage_distribution' : [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1],
  'tail_cpu_usage_distribution' : [0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99],
 },
]

instance_events = [
 {
  'time': 5,
  'collection_id': 1,
  'instance_index': 2,
  'scheduling_class' : 3,
  'collection_type' : 5, 
  'priority' : 6, 
  'alloc_collection_id' : 4, 
  'machine_id' : 3, 
  'alloc_instance_index' : 5,
  'resource_request': {'cpus': 0.6, 'memory':2}
 },
]

In [2]:
import apache_beam as beam
from apache_beam.io.gcp.internal.clients import bigquery
from simulator.pair_instance_event_to_usage import JoinUsageAndEvent
from simulator.map_to_schema import MapVMSampleToSchema
import json

with beam.Pipeline() as pipeline:
    input_usage = pipeline | "Create test dataset for usage" >> beam.Create(usages)
    input_event = pipeline | "Create test dataset for events" >> beam.Create(instance_events)
    
    usage_event_stream = JoinUsageAndEvent(input_usage, input_event) 
    final_table = usage_event_stream | "Map Joined Data to Schema" >> beam.Map(MapVMSampleToSchema) 
    
    table_spec = bigquery.TableReference(
        projectId='<input project ID>',
        datasetId='<input dataset name>',
        tableId='<input table name>')    
    
    f = open('table_schema.json')
    table_schema = json.loads(f.read())
    
    _ = final_table | beam.io.WriteToBigQuery(
        table_spec,
        schema=table_schema,
        write_disposition=beam.io.BigQueryDisposition.WRITE_TRUNCATE,
        create_disposition=beam.io.BigQueryDisposition.CREATE_IF_NEEDED)